In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_penalty_shoot-outs"
r = requests.get(url)
soup = bs(r.content, "lxml")

In [4]:
t = soup.find_all('table')

In [5]:
table = t[1]

In [6]:
rows = table.find_all('tr')

In [7]:
winners = []
for row in rows:
    cells = row.find_all('td')
    if cells:
        winners.append(cells[0].text[1:-1]) #spliting string as plain form is '\xa0West Germany\n'

winners

['West Germany',
 'France',
 'West Germany',
 'Belgium',
 'Republic of Ireland',
 'Argentina',
 'Argentina',
 'West Germany',
 'Bulgaria',
 'Sweden',
 'Brazil',
 'Argentina',
 'France',
 'Brazil',
 'Spain',
 'South Korea',
 'Ukraine',
 'Germany',
 'Portugal',
 'Italy',
 'Paraguay',
 'Uruguay',
 'Brazil',
 'Costa Rica',
 'Netherlands',
 'Argentina',
 'Russia',
 'Croatia',
 'England',
 'Croatia']

In [8]:
losers = []
for row in rows:
    cells = row.find_all('td')
    if cells:
        losers.append(cells[2].text[1:-1]) #spliting string as plain form is '\xa0West Germany\n'

losers

['France',
 'Brazil',
 'Mexico',
 'Spain',
 'Romania',
 'Yugoslavia',
 'Italy',
 'England',
 'Mexico',
 'Romania',
 'Italy',
 'England',
 'Italy',
 'Netherlands',
 'Republic of Ireland',
 'Spain',
 '\xa0Switzerland',
 'Argentina',
 'England',
 'France',
 'Japan',
 'Ghana',
 'Chile',
 'Greece',
 'Costa Rica',
 'Netherlands',
 'Spain',
 'Denmark',
 'Colombia',
 'Russia']

In [9]:
losers[losers.index('\xa0Switzerland')] = 'Switzerland'
losers

['France',
 'Brazil',
 'Mexico',
 'Spain',
 'Romania',
 'Yugoslavia',
 'Italy',
 'England',
 'Mexico',
 'Romania',
 'Italy',
 'England',
 'Italy',
 'Netherlands',
 'Republic of Ireland',
 'Spain',
 'Switzerland',
 'Argentina',
 'England',
 'France',
 'Japan',
 'Ghana',
 'Chile',
 'Greece',
 'Costa Rica',
 'Netherlands',
 'Spain',
 'Denmark',
 'Colombia',
 'Russia']